In [1]:
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import AutoPeftModelForCausalLM

In [2]:
def make_prompts(dataset, prompt_file="./prompts/prompt_v2.md"):
    with open(prompt_file, "r") as f:
        prompt = f.read()

    return prompt.format(user_question=dataset["question"], table_metadata_string=dataset["context"]).strip()

In [3]:
dataset = load_from_disk("./datasets/sql-create-context-split")["val"].take(2)
dataset[0]

{'answer': 'SELECT T3.Location, T2.Name FROM party_host AS T1 JOIN HOST AS T2 ON T1.Host_ID = T2.Host_ID JOIN party AS T3 ON T1.Party_ID = T3.Party_ID ORDER BY T2.Age',
 'question': 'Show the locations of parties and the names of the party hosts in ascending order of the age of the host.',
 'context': 'CREATE TABLE party (Location VARCHAR, Party_ID VARCHAR); CREATE TABLE HOST (Name VARCHAR, Host_ID VARCHAR, Age VARCHAR); CREATE TABLE party_host (Host_ID VARCHAR, Party_ID VARCHAR)'}

In [4]:
prompt = make_prompts(dataset[0])
prompt

'### Task\nGenerate a SQL query to answer [QUESTION]Show the locations of parties and the names of the party hosts in ascending order of the age of the host.[/QUESTION]\n\n### Instructions\n- End the SQL query with ";"\n- Do not explain the Answer SQL\n\n### Database Schema\nThe query will run on a database with the following schema:\nCREATE TABLE party (Location VARCHAR, Party_ID VARCHAR); CREATE TABLE HOST (Name VARCHAR, Host_ID VARCHAR, Age VARCHAR); CREATE TABLE party_host (Host_ID VARCHAR, Party_ID VARCHAR)\n\n### Answer\nGiven the database schema, here is the SQL query that answers [QUESTION]Show the locations of parties and the names of the party hosts in ascending order of the age of the host.[/QUESTION]\n[SQL]'

In [9]:
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")
tokenizer.pad_token_id = tokenizer.eos_token_id

In [27]:
model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-Instruct-hf",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,  # May not work on All GPU, use float16 instead if error
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=400,
    do_sample=False,
    num_beams=1,
)

outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [29]:
outputs[0]

'### Task\nGenerate a SQL query to answer [QUESTION]Show the locations of parties and the names of the party hosts in ascending order of the age of the host.[/QUESTION]\n\n### Instructions\n- End the SQL query with ";"\n- Do not explain the Answer SQL\n\n### Database Schema\nThe query will run on a database with the following schema:\nCREATE TABLE party (Location VARCHAR, Party_ID VARCHAR); CREATE TABLE HOST (Name VARCHAR, Host_ID VARCHAR, Age VARCHAR); CREATE TABLE party_host (Host_ID VARCHAR, Party_ID VARCHAR)\n\n### Answer\nGiven the database schema, here is the SQL query that answers [QUESTION]Show the locations of parties and the names of the party hosts in ascending order of the age of the host.[/QUESTION]\n[SQL]\nSELECT p.Location, h.Name\nFROM party p\nJOIN party_host ph ON p.Party_ID = ph.Party_ID\nJOIN HOST h ON ph.Host_ID = h.Host_ID\nORDER BY h.Age ASC;\n[/SQL]\n'

In [15]:
outputs[0].split("[SQL]")[-1].split(";")[0].strip()

'SELECT p.Location, h.Name\nFROM party p\nJOIN party_host ph ON p.Party_ID = ph.Party_ID\nJOIN HOST h ON ph.Host_ID = h.Host_ID\nORDER BY h.Age ASC'

In [16]:
dataset[0]['answer']

'SELECT T3.Location, T2.Name FROM party_host AS T1 JOIN HOST AS T2 ON T1.Host_ID = T2.Host_ID JOIN party AS T3 ON T1.Party_ID = T3.Party_ID ORDER BY T2.Age'

In [26]:
del model
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [7]:
tokenizer = AutoTokenizer.from_pretrained("./models/codellama-v1")

In [5]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "./models/codellama-v1",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=1024,
    do_sample=False,
    num_beams=1,
)

outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [9]:
outputs[0]

'### Task\nGenerate a SQL query to answer [QUESTION]Show the locations of parties and the names of the party hosts in ascending order of the age of the host.[/QUESTION]\n\n### Instructions\n- End the SQL query with ";"\n- Do not explain the Answer SQL\n\n### Database Schema\nThe query will run on a database with the following schema:\nCREATE TABLE party (Location VARCHAR, Party_ID VARCHAR); CREATE TABLE HOST (Name VARCHAR, Host_ID VARCHAR, Age VARCHAR); CREATE TABLE party_host (Host_ID VARCHAR, Party_ID VARCHAR)\n\n### Answer\nGiven the database schema, here is the SQL query that answers [QUESTION]Show the locations of parties and the names of the party hosts in ascending order of the age of the host.[/QUESTION]\n[SQL]SELECT T1.Location, T2.Name FROM party AS T1 JOIN party_host AS T3 ON T1.Party_ID = T3.Party_ID JOIN HOST AS T2 ON T3.Host_ID = T2.Host_ID ORDER BY T2.Age;[/SQL]\n\n### Explanation\nThe query joins the tables party, party_host, and HOST to get the locations of parties and

In [10]:
outputs[0].split("[SQL]")[-1].split(";")[0].strip()

'SELECT T1.Location, T2.Name FROM party AS T1 JOIN party_host AS T3 ON T1.Party_ID = T3.Party_ID JOIN HOST AS T2 ON T3.Host_ID = T2.Host_ID ORDER BY T2.Age'